## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

###### План работы:
- 1) Открытие файла с данными и изучение общуей информации.
- 2) Предобработка данных.
- 3) Ответим на следующие вопросы:
   - Есть ли зависимость между наличием детей и возвратом кредита в срок?
   - Есть ли зависимость между семейным положением и возвратом кредита в срок?
   - Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
   - Как разные цели кредита влияют на его возврат в срок?
- 4) Общий вывод.   

###   1) Открытие файла с данными и изучение общей информации. 

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
print(data.head(15))
print(data.info())
print(data.isnull().sum())

    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42               высшее             0   
1          1   -4024.803754         36              среднее             1   
2          0   -5623.422610         33              Среднее             1   
3          3   -4124.747207         32              среднее             1   
4          0  340266.072047         53              среднее             1   
5          0    -926.185831         27               высшее             0   
6          0   -2879.202052         43               высшее             0   
7          0    -152.779569         50              СРЕДНЕЕ             1   
8          2   -6929.865299         35               ВЫСШЕЕ             0   
9          0   -2188.756445         41              среднее             1   
10         2   -4171.483647         36               высшее             0   
11         0    -792.701887         40              среднее             1   

###### Вывод: 

Размер таблицы и количество данных достаточно большой чтобы можно было сделать определенные выводы.Количество строк составляет - 21525 почти по всем категориям, за исключением двух ( ежемесячный доход и общий трудовой стаж в днях - 19351).Это говорит нам о том что в таблице есть некорректные данные (пропуски).Количество пропусков равно 2174.Пропуски относятся к типу Nan.Так же в таблице есть отрицательные значения, и аномальные(слишком большие значения) значения. 

###  2) Предобработка данных.

### Обработка пропусков

###### Найдем среднее значение по каждой категории относительно рода занятости.

In [2]:
income_type_mean = data.groupby('income_type')['total_income', 'income_type'].mean()
print(income_type_mean)                 
                  

                  total_income
income_type                   
безработный      131339.751676
в декрете         53829.130729
госслужащий      170898.309923
компаньон        202417.461462
пенсионер        137127.465690
предприниматель  499163.144947
сотрудник        161380.260488
студент           98201.625314


###### Заменим все пропуска в столбце 'total_income' средним значением для каждой категории относительно рода занятости

In [3]:
data.loc[data['income_type']=='пенсионер', 'total_income'] = data.loc[data['income_type']=='пенсионер', 'total_income'].fillna(110179.690761)
data.loc[data['income_type']=='безработный', 'total_income'] = data.loc[data['income_type']=='безработный', 'total_income'].fillna(131339.751676)
data.loc[data['income_type']=='в декрете', 'total_income'] = data.loc[data['income_type']=='декрете', 'total_income'].fillna(53829.130729)
data.loc[data['income_type']=='госслужащий', 'total_income'] = data.loc[data['income_type']=='госслужащий', 'total_income'].fillna(139034.452386)
data.loc[data['income_type']=='компаньон', 'total_income'] = data.loc[data['income_type']=='компаньон', 'total_income'].fillna(162401.351555)
data.loc[data['income_type']=='предприниматель', 'total_income'] = data.loc[data['income_type']=='предприниматель', 'total_income'].fillna(249581.572474)   
data.loc[data['income_type']=='сотрудник', 'total_income'] = data.loc[data['income_type']=='сотрудник', 'total_income'].fillna(133546.457238)    
data.loc[data['income_type']=='студент', 'total_income'] = data.loc[data['income_type']=='студент', 'total_income'].fillna(98201.625314)    

###### Приведем к модулю значения в столбце 'days_employed', тем самым убрав отрицательные значения

In [4]:
data['days_employed'] = data['days_employed'].abs() 

###### Найдем медиану для и замениим на ее значение все пропуска в столбце 'days_employed'

In [5]:
data_median_days_employed = data['days_employed'].median()
data['days_employed'] = data['days_employed'].fillna(value=data_median_days_employed)

###### Заменим аномально большое количетсво дней стажа (превыщающее 20000 дней на значение медианы)

In [6]:
def too_much_days_employed(days):
    if days > 20000:
        return data_median_days_employed
    else:
        return days
data['days_employed'] = data['days_employed'].apply(too_much_days_employed)   
data['total_income'] = data['total_income'].fillna(value=0)
print(data.head(20))
print(data.info())
print(data.isnull().sum())

    children  days_employed  dob_years            education  education_id  \
0          1    8437.673028         42               высшее             0   
1          1    4024.803754         36              среднее             1   
2          0    5623.422610         33              Среднее             1   
3          3    4124.747207         32              среднее             1   
4          0    2194.220567         53              среднее             1   
5          0     926.185831         27               высшее             0   
6          0    2879.202052         43               высшее             0   
7          0     152.779569         50              СРЕДНЕЕ             1   
8          2    6929.865299         35               ВЫСШЕЕ             0   
9          0    2188.756445         41              среднее             1   
10         2    4171.483647         36               высшее             0   
11         0     792.701887         40              среднее             1   

###### Вывод 
- Сначала я вычислил средние значения для каждой категории граждан.И вставил их соответственно вместо каждого значени Nan в столбце "income_type".
- Далее в столбце "days_employed" я применил модуль, чтобы убрать отрицательные значения.Потом нашел медиану по этому же столбцу, так как в этом столбце есть аномально большие значения, и метод .median лучше подходит для нахождения среднего значения.
- Далее вставил вместо значения Nan и аномально больших значений, значение median по столбцу  "days_employed".В самом конце вызвал метод info() к таблице, и увидел что в столбце пристутствует одно значние Nan,поэтому пришлось заменить его на ''0''

### Замена типа данных

In [7]:
data['days_employed'] = data['days_employed'].astype(int)
data['total_income'] = data['total_income'].astype(int)
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB
None


### Вывод

В данном пункте перевел данные из типа данных типа float к типу данных integer.
Так как если переменная целочисленная – то имеет смысл перевести ее в целочисленный вид

### Обработка дубликатов

In [8]:
data['education'] = data['education'].str.lower()
data.duplicated()
data.drop_duplicates().reset_index(drop = True)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,2194,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21450,0,2194,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21451,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21452,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


### Вывод

Привел в столбце 'education' все ячейки к одному регистру.Потом посчитал количетво дубликатов - 71.Их возникнование по моему мнению связано с ошибками при заполнении и выгрузке данных.Так же допускаю, появление некоторых дубликатов при заполнении пропусков.

### Лемматизация

In [9]:
from pymystem3 import Mystem
m = Mystem()
values = ['свадьба','образование','автомобиль','недвижимость','жилье']
def unique_porpose(row):
    lemmas = m.lemmatize(row)
    for value in values:
        if value in lemmas:
            return value
        
data['unique_purpose'] = data['purpose'].apply(unique_porpose)
print(data.head(20))


    children  days_employed  dob_years            education  education_id  \
0          1           8437         42               высшее             0   
1          1           4024         36              среднее             1   
2          0           5623         33              среднее             1   
3          3           4124         32              среднее             1   
4          0           2194         53              среднее             1   
5          0            926         27               высшее             0   
6          0           2879         43               высшее             0   
7          0            152         50              среднее             1   
8          2           6929         35               высшее             0   
9          0           2188         41              среднее             1   
10         2           4171         36               высшее             0   
11         0            792         40              среднее             1   

### Вывод

 В данном пункте выделил леммы столбца 'purpose' ('свадьба','образование','автомобиль','недвижимость','жилье'), создав отдельный столбец 'unique_purpose'.Операции с недвижимостью и жильем разделил, так как по-моему мнению они разнятся.

### Категоризация данных

In [10]:
data['children'] = data['children'].abs()
def too_much_children(children):
    if children == 20:
        return children == 2
    else:
        return children
data['normal_counts_children'] = data['children'].apply(too_much_children)    


def financial_burden(row):
    x = row['normal_counts_children']
    y = row ['family_status_id']
    if x == 0:
        return 'легкая финансовая нагрузка'
    if x <=2 and y == 0 or y == 1:
        return 'средняя финансовая нагрузка'
    if x<=2 and y == 2 or y == 3 and y == 4:
        return 'тяжелая финансовая нагрузка'
    if x <= 3 and y == 0 or y == 1 :
        return 'тяжелая финансовая нагрузка'
    if x <= 3:
        return 'крайне тяжелая,критическая финансовая нагрузка'
data['financial_burden'] = data.apply(financial_burden,axis = 1)
print(data.head(20))

    children  days_employed  dob_years            education  education_id  \
0          1           8437         42               высшее             0   
1          1           4024         36              среднее             1   
2          0           5623         33              среднее             1   
3          3           4124         32              среднее             1   
4          0           2194         53              среднее             1   
5          0            926         27               высшее             0   
6          0           2879         43               высшее             0   
7          0            152         50              среднее             1   
8          2           6929         35               высшее             0   
9          0           2188         41              среднее             1   
10         2           4171         36               высшее             0   
11         0            792         40              среднее             1   

###### Вывод:
В данном пункте провел категоризацию по таким признакам как наличие детей, их количество и семейный статус.Создал новый столбец который присваивал конкретному человеку уровень финансовой нагрузки.

###  3) Ответим на следующие вопросы.

- 1) Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [11]:
data_pivot = data.pivot_table(index='normal_counts_children', columns = ['debt'], values = 'gender' ,  aggfunc='count')
data_pivot['ratio'] = data_pivot[1]/data_pivot[0]
print(data_pivot)

debt                          0       1     ratio
normal_counts_children                           
0                       13154.0  1071.0  0.081420
1                        4420.0   445.0  0.100679
2                        1861.0   194.0  0.104245
3                         303.0    27.0  0.089109
4                          37.0     4.0  0.108108
5                           9.0     NaN       NaN


###### Вывод

Как мы можем посмотреть по сводной таблице: не большая зависимости между наличием детей,их количеством и возвратом кредита в срок все же есть.Доля тех кто не вернул кредит в срок от тех кто это сделал (не имя детей) состовляет - 0,08.А доля тех кто  не вернул кредит в срок от тех кто это сделал имея детей - состовляет 0.1 (в независимоти от количества детей.   

- 2) Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [12]:
data_pivot = data.pivot_table(index='family_status', columns='debt', values = 'gender', aggfunc='count')
data_pivot['ratio'] = data_pivot[1]/data_pivot[0]
print(data_pivot)

debt                       0    1     ratio
family_status                              
Не женат / не замужем   2539  274  0.107917
в разводе               1110   85  0.076577
вдовец / вдова           897   63  0.070234
гражданский брак        3789  388  0.102402
женат / замужем        11449  931  0.081317


###### Вывод:
Между семейным положением и возвратом кредита в срок так же наблюдается зависимоть,более выраженная чем в предыдущем примере.Люди которые не состоят в официальном браке чаще не выполняют свои обязательства по возвращению кредита в срок по отношению к людям кто состоит в официальном браке, и еще меньше по отношению к людям кто разведен или тем кто потерял своего супруга/супругу.

- 3) Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Для этого разделим столбец с уровнем дохода на подгруппы (низкий,средний и высокий уровни дохода), и создадим новый столбец 
'income_level' - уровень дохода.Понему и сформируем сводную таблицу.

In [13]:
def income(money):
    if money <= 70000:
        return 'низкий доход'
    if money <= 130000:
        return 'средний доход'
    if money <= 180000:
        return 'высокий доход'
    
data['income_level'] = data['total_income'].apply(income) 
data_pivot = data.pivot_table(index='income_level', columns='debt', values='gender', aggfunc='count')
data_pivot['ratio'] = data_pivot[1]/data_pivot[0]
print(data_pivot)

debt              0    1     ratio
income_level                      
высокий доход  6165  581  0.094242
низкий доход   1373  101  0.073562
средний доход  6301  581  0.092208


###### Вывод 

Есть обратная зависимость от уровня дохода и возвратом кредита в срок.Как бы это не казалось странным, но люди с болеее низким доходом чаще возвращют кредит в срок по отношению к тем кто этого не делает вовремя.

- 4) Как разные цели кредита влияют на его возврат в срок?

In [14]:
data = data.pivot_table(index='unique_purpose', columns='debt', values='gender', aggfunc='count')
data['ratio'] = data[1]/data[0]
print(data)

debt               0    1     ratio
unique_purpose                     
автомобиль      3912  403  0.103016
жилье           4165  308  0.073950
недвижимость    5893  474  0.080434
образование     3652  370  0.101314
свадьба         2162  186  0.086031


### Вывод

Зависимость от цели кредита и его возврата в срок так же наблюдается, как и в первых двух вопросах касательно количества детей и семейного статуса.Как мы можем увидеть в таблице: люди в больщем количестве возвращают кредит в срок если это касается жилья или недвижимости, по отношению к тем кто этого не делает во время.

# Общий вывод

Проведя анализ данный, можно ответить на несколько ключевых вопросов:
- 1) При работе с данным были пропуска, неточности, дубликаты и аномальные значения
- 2) Из предыдущего пункта можно предположить что выгразка данных была проведенна с ошибками, а так же можно предположить
что ошибки были сделаны во время заполнения
- 3) И все же проведя работу с данными и приведя их в рабочий вид мы можем сделать некоторые выводы касательно зависимости между количеством детей, семейным статусом и возвратом кредита в срок:
    - На мой взгляд наличие, и количество детей не влияет навозврат кредита в срок.
    - Но что касается семейного статуса: тут можно удверждать что люди которые находят в браке, или состояли в нем или потеряли в связи со смертью своего супруга/супругу чаще отдают кредит в срок нежели те кто не состоит или не состоял в официальном браке. 